In [1]:
from src.features.simple_feature_calculator import SimpleFeatureCalculator
from research.model_pick.candle_fetch import FusionCandles
from strategies.BinanceBtcDeapV1Voting.models.config import FEAT_FRACDIFF, ALL_RAW_FEAT, SSMContainer
from jesse.helpers import date_to_timestamp
import lightgbm as lgb
import pandas as pd
import numpy as np

candle_container = FusionCandles(
    exchange="Binance Perpetual Futures", symbol="BTC-USDT", timeframe="1m"
)
candles = candle_container.get_candles("2025-01-01", "2025-10-12")

feat_calc = SimpleFeatureCalculator()
feat_calc.load(candles, sequential=True)
print("getting raw features...")
df_raw_feats = pd.DataFrame.from_dict(feat_calc.get(ALL_RAW_FEAT))
df_raw_feats.set_index(candles[:, 0].astype(int), inplace=True)
print(f"{df_raw_feats.shape = }")

model_deep_ssm = SSMContainer(model_type="deep_ssm")
model_lg_ssm = SSMContainer(model_type="lg_ssm")

df_deep_ssm = model_deep_ssm.transform(df_raw_feats[FEAT_FRACDIFF])
df_lg_ssm = model_lg_ssm.transform(df_raw_feats[FEAT_FRACDIFF])
df_feat_full = pd.concat([df_deep_ssm, df_lg_ssm, df_raw_feats], axis=1)

test_features = df_feat_full[
    (df_feat_full.index.to_numpy() >= date_to_timestamp("2025-03-01")) &
    (df_feat_full.index.to_numpy() < date_to_timestamp("2025-10-12"))
]

PyTorch configured: device=cpu, dtype=torch.float32


/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.12/site-packages/jesse/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


getting raw features...
df_raw_feats.shape = (2985, 8083)
Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.json
Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.json


In [2]:
import json

MODEL_TYPE = "c"
LAG = 4
PRED_NEXT = 1
THRESHOLD = 0.5
MODEL_NAME = f"{MODEL_TYPE}_L{LAG}_N{PRED_NEXT}"

with open("strategies/BinanceBtcDeapV1Voting/models/feature_info.json", "r") as f:
    feat_info = json.load(f)
df_selected_feats = test_features[feat_info[f"{MODEL_NAME}"]]

timestamps = candles[:, 0].astype(np.int64)
mask = (timestamps >= test_features.index[0]) & (timestamps <= test_features.index[-1])

close_price = candles[:, 2][mask]
datetimes = pd.to_datetime(test_features.index, unit="ms")
assert len(datetimes) == len(close_price)

model = lgb.Booster(model_file=f"strategies/BinanceBtcDeapV1Voting/models/model_{MODEL_NAME}.txt")
pred = model.predict(df_selected_feats)
assert len(pred) == len(datetimes)

In [4]:
from research.plot import analyze_confidence_slices

slice_res = analyze_confidence_slices(
    datetimes,
    pred,
    close_price,
    PRED_NEXT,
    output_dir=f"./temp/{MODEL_TYPE}_L{LAG}_N{PRED_NEXT}",
    granularity=0.02,
    lower_bound=0,
    upper_bound=1,
    threshold=0.5,
)


📊 时序对齐信息:
  - pred_next: 1 (预测未来第1根K线)
  - 原始数据长度: 2026
  - 对齐后数据长度: 2025
  - 丢弃的尾部数据: 1 条

已生成: slice_0.9800_1.0000_ratio_0.0040.jpg (切片区间: [0.9800, 1.0000] (做多), 样本占比: 0.40%)
已生成: slice_0.9600_0.9800_ratio_0.0286.jpg (切片区间: [0.9600, 0.9800] (做多), 样本占比: 2.86%)
已生成: slice_0.9400_0.9600_ratio_0.0444.jpg (切片区间: [0.9400, 0.9600] (做多), 样本占比: 4.44%)
已生成: slice_0.9200_0.9400_ratio_0.0321.jpg (切片区间: [0.9200, 0.9400] (做多), 样本占比: 3.21%)
已生成: slice_0.9000_0.9200_ratio_0.0410.jpg (切片区间: [0.9000, 0.9200] (做多), 样本占比: 4.10%)
已生成: slice_0.8800_0.9000_ratio_0.0306.jpg (切片区间: [0.8800, 0.9000] (做多), 样本占比: 3.06%)
已生成: slice_0.8600_0.8800_ratio_0.0262.jpg (切片区间: [0.8600, 0.8800] (做多), 样本占比: 2.62%)
已生成: slice_0.8400_0.8600_ratio_0.0257.jpg (切片区间: [0.8400, 0.8600] (做多), 样本占比: 2.57%)
已生成: slice_0.8200_0.8400_ratio_0.0257.jpg (切片区间: [0.8200, 0.8400] (做多), 样本占比: 2.57%)
已生成: slice_0.8000_0.8200_ratio_0.0198.jpg (切片区间: [0.8000, 0.8200] (做多), 样本占比: 1.98%)
已生成: slice_0.7800_0.8000_ratio_0.0202.jpg (切片区间: [0.7800,

In [ ]:
from strategies.BinanceBtcDeapV1Voting.models.config import LGBMContainer

model_container = LGBMContainer(
    model_type=MODEL_TYPE,
    lag=LAG,
    pred_next=PRED_NEXT,
    threshold=THRESHOLD,
)

In [ ]:
# TODO: 根据图片设定合适的filters


In [ ]:
model_container.save_filters()